首先看一个具有单隐藏层的多层感知机

In [1]:
import torch
from torch import nn

net = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    nn.Linear(8, 1)
)
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.1947],
        [-0.2753]], grad_fn=<AddmmBackward0>)

参数访问

In [2]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.2803, -0.3075, -0.0043,  0.1194,  0.2829,  0.0434, -0.0904, -0.2525]])), ('bias', tensor([-0.1319]))])


输出的结果告诉我们一些重要的事情： 首先，这个全连接层包含两个参数，分别是该层的权重和偏置。 两者都存储为单精度浮点数（float32）。 注意，参数名称允许唯一标识每个参数，即使在包含数百个层的网络中也是如此。

In [3]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1319], requires_grad=True)
tensor([-0.1319])


In [4]:
net[2].weight.grad == None

True

一次性访问所有参数
通过演示来比较访问第一个全连接层的参数和访问所有层

In [5]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


这为我们提供了另一种访问网络参数的方式

In [6]:
net.state_dict()['2.bias'].data

tensor([-0.1319])

从嵌套块收集参数

In [7]:
# 我们首先定义一个生成块的函数，然后将这些块组合到更大的块中
def block1():
    return nn.Sequential(
        nn.Linear(4, 8),
        nn.ReLU(),
        nn.Linear(8, 4),
        nn.ReLU()
    )
def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block{i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.1641],
        [-0.1641]], grad_fn=<AddmmBackward0>)

设计了网络后，我们看看它是如何工作的

In [8]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问他们。下面，我们访问第一个块中的第二个子块的第一层的偏置项。

In [9]:
rgnet[0][1][0].bias.data

tensor([ 0.0879, -0.0373, -0.0293, -0.0634, -0.0299, -0.4197,  0.2908, -0.4589])

正确的参数初始化

In [10]:
# 使用内置函数初始化
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0031,  0.0026,  0.0029, -0.0029]), tensor(0.))

In [11]:
# 我们还可以将所有参数初始化为给定的常数，比如初始化为1
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

我们还可以对某些块应用不同的初始化方法。例如，我们使用Xavier的方法初始化第一个神经网络层，然后将第三个神经网络层初始化为常量值为42。

In [12]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([0.0025, 0.1829, 0.2118, 0.0554])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


自定义初始化
同样，我们实现了一个函数来应用net

In [13]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape) for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000,  0.0000, -8.1885,  0.0000],
        [ 0.0000, -0.0000,  0.0000, -9.8891]], grad_fn=<SliceBackward0>)

注意：我们可以直接设置参数

In [16]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  3.0000, -5.1885,  3.0000])

参数绑定
有时我们希望在多个层之间共享参数：我们可以定义一个稠密层，然后使用他的参数来设置另一个层的参数。

In [18]:
# 我们需要给共享层一个名称，以便可以引用他的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(
    nn.Linear(4, 8),
    nn.ReLU(),
    shared,
    nn.ReLU(),
    shared,
    nn.ReLU(),
    nn.Linear(8, 1)
)
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
